# Task 2 (Interpolation) Analysis - Kernel Comparison

This notebook analyzes the performance of different GP kernels on **Task 2 (Interpolation)**:

## Task Description
- **Task 2 (Interpolation)**: Predict **intermediate views** within the training range
- **Train Views**: Even-indexed views (0°, 40°, 80°, 120°, 160°, 200°, 240°, 280°, 320°)
- **Val/Test Views**: Odd-indexed views (20°, 60°, 100°, 140°, 180°, 220°, 260°, 300°, 340°)
- **Goal**: Test in-range generalization via GP interpolation

## Research Question
> "Does imposing smoothness via structured kernels improve performance when test views lie **within** the training range?"

## Kernels Compared
1. **Full Rank**: Free-form learnable covariance (Q×Q parameters)
2. **Periodic**: Standard periodic kernel with learned lengthscale
3. **SM Wrapped**: Spectral Mixture with wrapped lag distance
4. **SM Free**: Spectral Mixture (unwrapped)

## Analysis (Extended for Interpolation)
- Load best checkpoints from each kernel (5 seeds each)
- Evaluate on test set (intermediate views)
- **Per-View MSE**: Analyze how well each kernel interpolates at different angles
- **Distance-to-Nearest-Train Analysis**: MSE vs distance to nearest training view
- Visualize kernel matrices and reconstructions

## 1. Setup

In [ ]:
import os
import sys
import glob
import pickle
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
import os
import sys

current_dir = os.getcwd()
print(f"📍 Current directory: {current_dir}")

if current_dir == '/content':
    print("\n🔄 Mounting Google Drive...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = '/content/drive/MyDrive/gppvae'
        if os.path.exists(drive_path):
            PROJECT_PATH = drive_path
            print(f"✅ Found project in Google Drive: {PROJECT_PATH}")
        else:
            print(f"⚠️ Project not found at: {drive_path}")
            PROJECT_PATH = '/content'
    except Exception as e:
        print(f"Could not mount Drive: {e}")
        PROJECT_PATH = '/content'
else:
    if 'notebooks' in current_dir:
        PROJECT_PATH = os.path.dirname(os.path.dirname(current_dir))
    else:
        PROJECT_PATH = current_dir
    print(f"💻 Using project path: {PROJECT_PATH}")

# Add code paths
coil100_path = os.path.join(PROJECT_PATH, 'GPPVAE/pysrc/coil100')
sys.path.insert(0, coil100_path)

# Change to project root
os.chdir(PROJECT_PATH)
print(f"Working directory: {os.getcwd()}")
print(f"Code path added: {coil100_path}")

In [ ]:
# Import models
from vae import FaceVAE
from vmod import Vmodel
from gp import GP
from data_parser import COIL100Dataset, get_n_views, get_num_objects

print("✅ All modules imported")

## 2. Configuration

In [ ]:
# Task 2 (Interpolation) configuration
CONFIG = {
    'task': 'task2_interpolation',
    'data_path': './data/coil100/coil100_task2_interpolation.h5',
    'batch_size': 64,
    'xdim': 64,  # Object embedding dimension
}

# Auto-detect results folder location
if os.path.exists('./GPPVAE/results'):
    CONFIG['results_base'] = './GPPVAE/results'
    print(f"✅ Found results at: {os.path.abspath(CONFIG['results_base'])}")
elif os.path.exists('./results'):
    CONFIG['results_base'] = './results'
    print(f"✅ Found results at: {os.path.abspath(CONFIG['results_base'])}")
else:
    CONFIG['results_base'] = './results'
    print(f"⚠️ Results folder not found, using default: {CONFIG['results_base']}")

# Auto-detect data folder location
if os.path.exists('./GPPVAE/data/coil100/coil100_task2_interpolation.h5'):
    CONFIG['data_path'] = './GPPVAE/data/coil100/coil100_task2_interpolation.h5'
    print(f"✅ Found data at: {os.path.abspath(CONFIG['data_path'])}")
elif os.path.exists('./data/coil100/coil100_task2_interpolation.h5'):
    CONFIG['data_path'] = './data/coil100/coil100_task2_interpolation.h5'
    print(f"✅ Found data at: {os.path.abspath(CONFIG['data_path'])}")
else:
    CONFIG['data_path'] = './data/coil100/coil100_task2_interpolation.h5'
    print(f"⚠️ Data file not found, using default: {CONFIG['data_path']}")

# View configuration for Task 2 (Interpolation)
# Train: even views (0, 2, 4, ..., 16) = 0°, 40°, 80°, ..., 320°
# Test: odd views (1, 3, 5, ..., 17) = 20°, 60°, 100°, ..., 340°
TRAIN_VIEW_INDICES = [0, 2, 4, 6, 8, 10, 12, 14, 16]  # Even indices
TEST_VIEW_INDICES = [1, 3, 5, 7, 9, 11, 13, 15, 17]   # Odd indices (interpolation targets)
VIEW_ANGLES = {i: i * 20 for i in range(18)}  # View index to angle mapping

print(f"\n📊 Interpolation Task Setup:")
print(f"   Train views: {[VIEW_ANGLES[v] for v in TRAIN_VIEW_INDICES]}°")
print(f"   Test views:  {[VIEW_ANGLES[v] for v in TEST_VIEW_INDICES]}°")

# Kernel configurations (must match training)
KERNEL_CONFIGS = {
    'fullrank': {
        'folder': 'task2_fullrank',
        'view_kernel': 'full_rank',
        'kernel_kwargs': {},
        'display_name': 'Full Rank',
        'color': '#e74c3c',
    },
    'periodic': {
        'folder': 'task2_periodic',
        'view_kernel': 'periodic',
        'kernel_kwargs': {'period': 360.0, 'lengthscale': 1.0, 'variance': 1.0},
        'display_name': 'Periodic',
        'color': '#3498db',
    },
    'sm_wrapped': {
        'folder': 'task2_sm_wrapped',
        'view_kernel': 'sm_circle',
        'kernel_kwargs': {'freq_init': [1/360.0, 1/40.0], 'weight_init':[0.5, 0.5], 'length_init':[90,30]},
        'display_name': 'SM (Wrapped)',
        'color': '#2ecc71',
    },
    'sm_free': {
        'folder': 'task2_sm_free',
        'view_kernel': 'sm_circle',
        'kernel_kwargs': {'freq_init': [1/360.0, 1/40.0], 'weight_init':[0.5, 0.5], 'use_angle_input': True},
        'display_name': 'SM (Free)',
        'color': '#9b59b6',
    },
}

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 3. Load Data

In [ ]:
# Load datasets
train_data = COIL100Dataset(CONFIG['data_path'], split='train', use_angle_encoding=False)
val_data = COIL100Dataset(CONFIG['data_path'], split='val', use_angle_encoding=False)
test_data = COIL100Dataset(CONFIG['data_path'], split='test', use_angle_encoding=False)

train_queue = DataLoader(train_data, batch_size=CONFIG['batch_size'], shuffle=False)
val_queue = DataLoader(val_data, batch_size=CONFIG['batch_size'], shuffle=False)
test_queue = DataLoader(test_data, batch_size=CONFIG['batch_size'], shuffle=False)

# Get dimensions
P = get_num_objects(CONFIG['data_path'])  # Number of objects
Q = get_n_views()  # Number of views (18)

print(f"\n📊 Task 2 (Interpolation) Dataset:")
print(f"   Objects (P): {P}")
print(f"   Views (Q): {Q}")
print(f"   Train: {len(train_data)} samples")
print(f"   Val: {len(val_data)} samples")
print(f"   Test: {len(test_data)} samples")

# Analyze view distribution
train_views = set(train_data.Rid.numpy())
test_views = set(test_data.Rid.numpy())
print(f"\n   Train view indices: {sorted(train_views)}")
print(f"   Test view indices: {sorted(test_views)}")
print(f"   Train angles: {[VIEW_ANGLES.get(int(v), v*20) for v in sorted(train_views)]}°")
print(f"   Test angles: {[VIEW_ANGLES.get(int(v), v*20) for v in sorted(test_views)]}°")

# Create tensors for indices
Dt = Variable(train_data.Did.long(), requires_grad=False).to(device)
Wt = Variable(train_data.Rid.long(), requires_grad=False).to(device)
Dtest = Variable(test_data.Did.long(), requires_grad=False).to(device)
Wtest = Variable(test_data.Rid.long(), requires_grad=False).to(device)

## 4. Helper Functions

In [ ]:
def find_run_folders(results_base, kernel_folder):
    """Find all run folders (seeds) for a kernel."""
    kernel_path = os.path.join(results_base, kernel_folder)
    if not os.path.exists(kernel_path):
        print(f"⚠️ Kernel folder not found: {kernel_path}")
        return []
    
    runs = sorted([d for d in os.listdir(kernel_path) 
                   if os.path.isdir(os.path.join(kernel_path, d))])
    return [os.path.join(kernel_path, r) for r in runs]


def load_vae_config(run_folder=None):
    """Load VAE config from GPPVAE folder structure or use default."""
    possible_paths = [
        './GPPVAE/pysrc/coil100/vae.cfg.p',
        './out/vae/vae.cfg.p',
        './out/vae_colab/*/vae.cfg.p',
    ]
    
    for path_pattern in possible_paths:
        if '*' in path_pattern:
            matches = glob.glob(path_pattern)
            if matches:
                try:
                    with open(matches[0], 'rb') as f:
                        vae_cfg = pickle.load(f)
                    return vae_cfg
                except:
                    continue
        elif os.path.exists(path_pattern):
            try:
                with open(path_pattern, 'rb') as f:
                    vae_cfg = pickle.load(f)
                return vae_cfg
            except:
                continue
    
    # Default VAE config for COIL-100
    vae_cfg = {
        'img_size': 128,
        'nf': 32,
        'zdim': 256,
        'steps': 5,
        'colors': 3,
        'act': 'elu',
        'vy': 0.001
    }
    return vae_cfg


def load_models(run_folder, kernel_config, P, Q, xdim, device):
    """Load VAE, Vmodel, and GP from a run folder's best checkpoint."""
    weights_dir = os.path.join(run_folder, 'weights')
    gp_weights_path = os.path.join(weights_dir, 'gp_weights.best.pt')
    vae_weights_path = os.path.join(weights_dir, 'vae_weights.best.pt')
    
    if not os.path.exists(gp_weights_path):
        raise FileNotFoundError(f"GP weights not found: {gp_weights_path}")
    if not os.path.exists(vae_weights_path):
        raise FileNotFoundError(f"VAE weights not found: {vae_weights_path}")
    
    # Load VAE
    vae_cfg = load_vae_config(run_folder)
    vae = FaceVAE(**vae_cfg).to(device)
    vae.load_state_dict(torch.load(vae_weights_path, map_location=device))
    vae.eval()
    
    # Load Vmodel and GP
    vm = Vmodel(
        P=P, Q=Q, p=xdim,
        view_kernel=kernel_config['view_kernel'],
        **kernel_config['kernel_kwargs']
    ).to(device)
    gp = GP(n_rand_effs=1).to(device)
    
    checkpoint = torch.load(gp_weights_path, map_location=device)
    gp.load_state_dict(checkpoint['gp_state'])
    vm.load_state_dict(checkpoint['vm_state'])
    
    vm.eval()
    gp.eval()
    
    return vae, vm, gp


def encode_dataset(vae, data_queue, device):
    """Encode all images in a dataset to latent space."""
    vae.eval()
    n = data_queue.dataset.Y.shape[0]
    zdim = 256
    
    Zm = torch.zeros(n, zdim).to(device)
    
    with torch.no_grad():
        for data in data_queue:
            y = data[0].to(device)
            idxs = data[-1].to(device)
            zm, _ = vae.encode(y)
            Zm[idxs] = zm.detach()
    
    return Zm


def evaluate_on_test_with_per_view(vae, vm, gp, train_queue, test_queue, 
                                    Dt, Wt, Dtest, Wtest, device):
    """
    Evaluate model on test set with per-view MSE breakdown.
    
    Returns:
        mse_test: Mean MSE on test set
        mse_per_sample: MSE for each test sample
        mse_per_view: Dict mapping view index to list of MSEs
    """
    vae.eval()
    vm.eval()
    gp.eval()
    
    with torch.no_grad():
        # Encode training data
        Zm = encode_dataset(vae, train_queue, device)
        
        # Compute V matrices
        Vt = vm(Dt, Wt).detach()
        Vtest = vm(Dtest, Wtest).detach()
        
        # GP prediction
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        Zo_test = vs[0] * Vtest.mm(Vt.transpose(0, 1).mm(Kiz))
        
        # Compute MSE per sample and per view
        test_Rid = test_queue.dataset.Rid
        mse_per_view = {}
        mse_per_sample = []
        mse_test_total = 0.0
        
        for data in test_queue:
            idxs = data[-1].to(device)
            Ytest = data[0].to(device)
            Yo = vae.decode(Zo_test[idxs])
            mse_batch = ((Ytest - Yo) ** 2).view(Ytest.shape[0], -1).mean(1)
            
            for i, idx in enumerate(data[-1]):
                view = int(test_Rid[idx].item())
                mse_val = mse_batch[i].item()
                
                if view not in mse_per_view:
                    mse_per_view[view] = []
                mse_per_view[view].append(mse_val)
                mse_per_sample.append(mse_val)
            
            mse_test_total += mse_batch.sum().item()
        
        mse_test = mse_test_total / len(test_queue.dataset)
    
    return mse_test, np.array(mse_per_sample), mse_per_view


def get_reconstructions(vae, vm, gp, train_queue, test_queue,
                        Dt, Wt, Dtest, Wtest, device, n_samples=24):
    """Get sample reconstructions for visualization."""
    vae.eval()
    vm.eval()
    gp.eval()
    
    with torch.no_grad():
        Zm = encode_dataset(vae, train_queue, device)
        Vt = vm(Dt, Wt).detach()
        Vtest = vm(Dtest, Wtest).detach()
        
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        Zo_test = vs[0] * Vtest.mm(Vt.transpose(0, 1).mm(Kiz))
        
        n_total = len(test_queue.dataset)
        sample_stride = max(1, n_total // n_samples)
        sample_indices = list(range(0, n_total, sample_stride))[:n_samples]
        
        Y_orig = test_queue.dataset.Y[sample_indices].numpy().transpose(0, 2, 3, 1)
        sample_indices_tensor = torch.tensor(sample_indices, dtype=torch.long).to(device)
        Y_recon = vae.decode(Zo_test[sample_indices_tensor])
        Y_recon = Y_recon.cpu().numpy().transpose(0, 2, 3, 1)
    
    return Y_orig, Y_recon


def compute_distance_to_nearest_train(test_view_idx, train_view_indices):
    """Compute angular distance from test view to nearest training view."""
    test_angle = test_view_idx * 20
    train_angles = [v * 20 for v in train_view_indices]
    
    min_dist = float('inf')
    for train_angle in train_angles:
        # Circular distance
        dist = min(abs(test_angle - train_angle), 360 - abs(test_angle - train_angle))
        min_dist = min(min_dist, dist)
    return min_dist

print("✅ Helper functions defined")

## 5. Evaluate All Kernels

In [ ]:
# Results storage
results = {}

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    print(f"\n{'='*60}")
    print(f"Evaluating: {kernel_config['display_name']}")
    print(f"{'='*60}")
    
    run_folders = find_run_folders(CONFIG['results_base'], kernel_config['folder'])
    
    if not run_folders:
        print(f"⚠️ No runs found for {kernel_name}")
        continue
    
    print(f"Found {len(run_folders)} runs (seeds)")
    
    kernel_results = {
        'mse_per_seed': [],
        'mse_per_sample_all': [],
        'mse_per_view_all': [],  # Per-view breakdown for each seed
        'kernel_matrices': [],
        'variance_ratios': [],
    }
    
    for i, run_folder in enumerate(run_folders):
        try:
            print(f"  Seed {i}: {os.path.basename(run_folder)}...", end=" ")
            
            vae, vm, gp = load_models(
                run_folder, kernel_config, P, Q, 
                CONFIG['xdim'], device
            )
            
            mse_test, mse_per_sample, mse_per_view = evaluate_on_test_with_per_view(
                vae, vm, gp, train_queue, test_queue,
                Dt, Wt, Dtest, Wtest, device
            )
            
            with torch.no_grad():
                K = vm.get_kernel_matrix().cpu().numpy()
                vs = gp.get_vs().cpu().numpy()
                variance_ratio = vs[0] / (vs[0] + vs[1])
            
            kernel_results['mse_per_seed'].append(mse_test)
            kernel_results['mse_per_sample_all'].append(mse_per_sample)
            kernel_results['mse_per_view_all'].append(mse_per_view)
            kernel_results['kernel_matrices'].append(K)
            kernel_results['variance_ratios'].append(variance_ratio)
            
            print(f"MSE = {mse_test:.6f}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
            continue
    
    if kernel_results['mse_per_seed']:
        mse_array = np.array(kernel_results['mse_per_seed'])
        kernel_results['mean_mse'] = np.mean(mse_array)
        kernel_results['std_mse'] = np.std(mse_array)
        kernel_results['var_mse'] = np.var(mse_array)
        
        # Aggregate per-view MSE across seeds
        all_views = set()
        for pv in kernel_results['mse_per_view_all']:
            all_views.update(pv.keys())
        
        kernel_results['mean_mse_per_view'] = {}
        kernel_results['std_mse_per_view'] = {}
        for view in all_views:
            view_mses = []
            for pv in kernel_results['mse_per_view_all']:
                if view in pv:
                    view_mses.extend(pv[view])
            kernel_results['mean_mse_per_view'][view] = np.mean(view_mses)
            kernel_results['std_mse_per_view'][view] = np.std(view_mses)
        
        print(f"\n  📊 {kernel_config['display_name']} Summary:")
        print(f"     Mean MSE: {kernel_results['mean_mse']:.6f} ± {kernel_results['std_mse']:.6f}")
        print(f"     Seeds: {len(kernel_results['mse_per_seed'])}")
    
    results[kernel_name] = kernel_results

print("\n" + "="*60)
print("✅ Evaluation complete!")

## 6. Summary Table

In [ ]:
# Create summary DataFrame
summary_data = []

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and results[kernel_name]['mse_per_seed']:
        r = results[kernel_name]
        summary_data.append({
            'Kernel': kernel_config['display_name'],
            'Mean MSE': r['mean_mse'],
            'Std MSE': r['std_mse'],
            'Var MSE': r['var_mse'],
            'Min MSE': np.min(r['mse_per_seed']),
            'Max MSE': np.max(r['mse_per_seed']),
            'N Seeds': len(r['mse_per_seed']),
            'Mean Var Ratio': np.mean(r['variance_ratios']),
        })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Mean MSE')

print("\n📊 Task 2 (Interpolation) - Kernel Comparison Summary")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)

In [ ]:
# Display formatted table
styled_df = summary_df.style.format({
    'Mean MSE': '{:.6f}',
    'Std MSE': '{:.6f}',
    'Var MSE': '{:.8f}',
    'Min MSE': '{:.6f}',
    'Max MSE': '{:.6f}',
    'Mean Var Ratio': '{:.3f}',
}).background_gradient(subset=['Mean MSE'], cmap='RdYlGn_r')

styled_df

## 7. Visualization: MSE Comparison

In [ ]:
# Bar plot with error bars - clean style
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

kernel_names = []
means = []
stds = []

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and results[kernel_name]['mse_per_seed']:
        kernel_names.append(kernel_config['display_name'])
        means.append(results[kernel_name]['mean_mse'])
        stds.append(results[kernel_name]['std_mse'])

x = np.arange(len(kernel_names))
bars = ax.bar(x, means, yerr=stds, capsize=5, color='#808080', alpha=0.8, edgecolor='black', linewidth=0.5)
ax.set_xticks(x)
ax.set_xticklabels(kernel_names, rotation=0, ha='center')
ax.set_ylabel('MSE [test set]')
ax.set_ylim(bottom=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)

plt.tight_layout()
plt.savefig('./notebooks/analysis/task2_mse_bar.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Saved: task2_mse_bar.png (300 DPI)")

# Box plot - clean style
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

data_for_box = []
labels_for_box = []

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and results[kernel_name]['mse_per_seed']:
        data_for_box.append(results[kernel_name]['mse_per_seed'])
        labels_for_box.append(kernel_config['display_name'])

bp = ax.boxplot(data_for_box, labels=labels_for_box, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('#808080')
    patch.set_alpha(0.7)
    patch.set_linewidth(0.5)

for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
    plt.setp(bp[element], linewidth=0.5)

ax.set_ylabel('MSE [test set]')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)
ax.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('./notebooks/analysis/task2_mse_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Saved: task2_mse_boxplot.png (300 DPI)")

## 8. Per-View MSE Analysis (Interpolation Quality)

This is the key analysis for Task 2: How well does each kernel interpolate to intermediate views?

In [ ]:
# Per-view MSE comparison plots
test_views_sorted = sorted([int(v) for v in test_views])

# Define consistent colors for each kernel
line_colors = {
    'Full Rank': '#1f77b4',      # Blue
    'Periodic': '#ff7f0e',        # Orange
    'SM (Wrapped)': '#9467bd',    # Purple
    'SM (Free)': '#8c564b',       # Brown
}

# Bar plot: Per-view MSE
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

n_kernels = len([k for k in KERNEL_CONFIGS if k in results and 'mean_mse_per_view' in results[k]])
bar_width = 0.8 / n_kernels
x_positions = np.arange(len(test_views_sorted))

for i, (kernel_name, kernel_config) in enumerate(KERNEL_CONFIGS.items()):
    if kernel_name in results and 'mean_mse_per_view' in results[kernel_name]:
        mean_per_view = results[kernel_name]['mean_mse_per_view']
        mses = [mean_per_view.get(v, 0) for v in test_views_sorted]
        display_name = kernel_config['display_name']
        color = line_colors.get(display_name, kernel_config['color'])
        ax.bar(x_positions + i * bar_width, mses, bar_width, 
               label=display_name, color=color, alpha=0.8)

ax.set_xlabel('Test View Angle')
ax.set_ylabel('Mean MSE')
ax.set_xticks(x_positions + bar_width * 1.5)
ax.set_xticklabels([f"{int(v)*20}°" for v in test_views_sorted], rotation=45)
ax.legend()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)

plt.tight_layout()
plt.savefig('./notebooks/analysis/task2_per_view_bar.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Saved: task2_per_view_bar.png (300 DPI)")

# Line plot showing interpolation pattern
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and 'mean_mse_per_view' in results[kernel_name]:
        mean_per_view = results[kernel_name]['mean_mse_per_view']
        angles = [v * 20 for v in sorted(mean_per_view.keys())]
        mses = [mean_per_view[v] for v in sorted(mean_per_view.keys())]
        display_name = kernel_config['display_name']
        color = line_colors.get(display_name, kernel_config['color'])
        ax.plot(angles, mses, 'o-', label=display_name, 
                color=color, markersize=8, linewidth=2)

# Mark training views
train_angles = [v * 20 for v in train_view_set]
for angle in train_angles:
    ax.axvline(x=angle, color='gray', linestyle='--', alpha=0.3)

ax.set_xlabel('View Angle (°)')
ax.set_ylabel('Mean MSE')
ax.legend(loc='best')
ax.set_xlim(-10, 350)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(False)

plt.tight_layout()
plt.savefig('./notebooks/analysis/task2_per_view_line.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Saved: task2_per_view_line.png (300 DPI)")

In [ ]:
# Per-view MSE table
per_view_data = []
test_views_sorted = sorted([int(v) for v in test_views])

for view in test_views_sorted:
    row = {'View': f"{view*20}°", 'View Index': view}
    for kernel_name, kernel_config in KERNEL_CONFIGS.items():
        if kernel_name in results and 'mean_mse_per_view' in results[kernel_name]:
            row[kernel_config['display_name']] = results[kernel_name]['mean_mse_per_view'].get(view, np.nan)
    per_view_data.append(row)

per_view_df = pd.DataFrame(per_view_data)
print("\n📊 Per-View MSE Breakdown (Interpolation Targets):")
print("="*80)
print(per_view_df.to_string(index=False))

# Find best kernel per view
print("\n🏆 Best Kernel per View:")
kernel_cols = [c for c in per_view_df.columns if c not in ['View', 'View Index']]
for _, row in per_view_df.iterrows():
    best_kernel = min(kernel_cols, key=lambda k: row[k] if pd.notna(row[k]) else float('inf'))
    print(f"   {row['View']}: {best_kernel} ({row[best_kernel]:.6f})")

## 9. Interpolation Smoothness Analysis

Analyze how smoothly each kernel interpolates between training views.

In [ ]:
# Analyze interpolation as a function of distance to nearest training view
# For interpolation task, all test views are 20° away from nearest train views

train_view_set = set(train_data.Rid.numpy().astype(int))

print("📊 Interpolation Distance Analysis:")
print("="*60)

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and 'mean_mse_per_view' in results[kernel_name]:
        mean_per_view = results[kernel_name]['mean_mse_per_view']
        
        # Compute average MSE
        avg_mse = np.mean(list(mean_per_view.values()))
        std_mse = np.std(list(mean_per_view.values()))
        
        # Variance across views (uniformity of interpolation)
        view_variance = np.var(list(mean_per_view.values()))
        
        print(f"\n{kernel_config['display_name']}:")
        print(f"   Mean MSE across views: {avg_mse:.6f} ± {std_mse:.6f}")
        print(f"   Variance across views: {view_variance:.8f} (lower = more uniform interpolation)")

## 10. Kernel Matrices

In [ ]:
# Plot kernel matrices (matching callback_gppvae style)
import pylab as pl
pl.rcdefaults()

n_kernels = len([k for k in results if results[k]['kernel_matrices']])
if n_kernels > 0:
    fig, axes = pl.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()

    ax_idx = 0
    for kernel_name, kernel_config in KERNEL_CONFIGS.items():
        if kernel_name in results and results[kernel_name]['kernel_matrices']:
            K = results[kernel_name]['kernel_matrices'][0]

            ax = axes[ax_idx]
            im = ax.imshow(K, vmin=-0.4, vmax=1, aspect='auto')

            tick_positions = [0, 4, 8, 12, 17]
            tick_labels = [f"{p*20}°" for p in tick_positions]
            ax.set_xticks(tick_positions)
            ax.set_xticklabels(tick_labels, fontsize=8)
            ax.set_yticks(tick_positions)
            ax.set_yticklabels(tick_labels, fontsize=8)
            ax.tick_params(labelsize=8)
            ax.set_title(kernel_config['display_name'], fontsize=10)

            pl.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            ax_idx += 1

    # Hide unused subplots
    for i in range(ax_idx, 4):
        axes[i].axis('off')

    pl.tight_layout()
    pl.savefig('./notebooks/analysis/task2_kernel_matrices.png', dpi=300, bbox_inches='tight')
    pl.show()
    print("📊 Saved: task2_kernel_matrices.png (300 DPI)")
else:
    print("⚠️ No kernel matrices to display")

## 11. Sample Reconstructions

In [ ]:
# Get reconstructions from best seed of each kernel
n_samples = 8
n_kernels = len([k for k in results if results[k]['mse_per_seed']])

if n_kernels > 0:
    fig, axes = plt.subplots(n_kernels + 1, n_samples, figsize=(n_samples * 1.5, (n_kernels + 1) * 1.5))
    
    # Ground truth
    n_total = len(test_data)
    sample_stride = max(1, n_total // n_samples)
    sample_indices = list(range(0, n_total, sample_stride))[:n_samples]
    Y_gt = test_data.Y[sample_indices].numpy().transpose(0, 2, 3, 1)
    
    for i in range(n_samples):
        axes[0, i].imshow(np.clip(Y_gt[i], 0, 1))
        axes[0, i].axis('off')
        # Show view angle
        view_idx = int(test_data.Rid[sample_indices[i]].item())
        axes[0, i].set_title(f"{view_idx*20}°", fontsize=8)
    axes[0, 0].set_ylabel('GT', fontsize=10)
    
    row_idx = 1
    for kernel_name, kernel_config in KERNEL_CONFIGS.items():
        if kernel_name in results and results[kernel_name]['mse_per_seed']:
            best_seed_idx = np.argmin(results[kernel_name]['mse_per_seed'])
            best_run = find_run_folders(CONFIG['results_base'], kernel_config['folder'])[best_seed_idx]
            
            vae, vm, gp = load_models(best_run, kernel_config, P, Q, CONFIG['xdim'], device)
            Y_orig, Y_recon = get_reconstructions(
                vae, vm, gp, train_queue, test_queue,
                Dt, Wt, Dtest, Wtest, device, n_samples=n_samples
            )
            
            for i in range(n_samples):
                axes[row_idx, i].imshow(np.clip(Y_recon[i], 0, 1))
                axes[row_idx, i].axis('off')
            
            mse = results[kernel_name]['mse_per_seed'][best_seed_idx]
            axes[row_idx, 0].set_ylabel(f"{kernel_config['display_name']}\n({mse:.4f})", fontsize=8)
            row_idx += 1
    
    plt.tight_layout()
    plt.savefig('./notebooks/analysis/task2_reconstructions.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("📊 Saved: task2_reconstructions.png (300 DPI)")

## 12. Statistical Analysis

In [ ]:
from scipy import stats

kernel_names_list = [k for k in KERNEL_CONFIGS if k in results and results[k]['mse_per_seed']]

if len(kernel_names_list) >= 2:
    print("\n📊 Pairwise T-Tests (p-values)")
    print("="*60)
    
    for i, k1 in enumerate(kernel_names_list):
        for j, k2 in enumerate(kernel_names_list):
            if i < j:
                mse1 = results[k1]['mse_per_seed']
                mse2 = results[k2]['mse_per_seed']
                
                t_stat, p_value = stats.ttest_ind(mse1, mse2)
                
                k1_name = KERNEL_CONFIGS[k1]['display_name']
                k2_name = KERNEL_CONFIGS[k2]['display_name']
                sig = "*" if p_value < 0.05 else ""
                print(f"{k1_name} vs {k2_name}: p = {p_value:.4f} {sig}")
    
    print("\n* indicates p < 0.05 (statistically significant)")

## 13. Callback-Style Plots

In [ ]:
from callbacks import callback_gppvae, _compose_multi
import pylab as pl

def generate_callback_plot_for_kernel(kernel_name, kernel_config, results, 
                                       train_queue, test_queue,
                                       Dt, Wt, Dtest, Wtest, P, Q, xdim, device,
                                       output_file):
    if kernel_name not in results or not results[kernel_name]['mse_per_seed']:
        return None
    
    pl.rcdefaults()
    
    best_seed_idx = np.argmin(results[kernel_name]['mse_per_seed'])
    best_run = find_run_folders(CONFIG['results_base'], kernel_config['folder'])[best_seed_idx]
    
    vae, vm, gp = load_models(best_run, kernel_config, P, Q, xdim, device)
    
    with torch.no_grad():
        X = vm.x().cpu().numpy()
        W = vm.v().cpu().numpy()
        XX = X @ X.T
        WW = W @ W.T
        covs = {"XX": XX[:20, :20], "WW": WW}
        
        Zm = encode_dataset(vae, train_queue, device)
        Vt = vm(Dt, Wt).detach()
        Vtest = vm(Dtest, Wtest).detach()
        
        vs = gp.get_vs()
        U, UBi, _ = gp.U_UBi_Shb([Vt], vs)
        Kiz = gp.solve(Zm, U, UBi, vs)
        Zo_test = vs[0] * Vtest.mm(Vt.transpose(0, 1).mm(Kiz))
        
        n_total = len(test_queue.dataset)
        sample_stride = max(1, n_total // 24)
        sample_indices = np.arange(0, n_total, sample_stride)[:24]
        sample_indices_tensor = torch.tensor(sample_indices, dtype=torch.long).to(device)
        
        Yv = test_queue.dataset.Y[sample_indices].numpy().transpose((0, 2, 3, 1))
        Y_input = test_queue.dataset.Y[sample_indices].to(device)
        Zm_test, _ = vae.encode(Y_input)
        Yr = vae.decode(Zm_test).data.cpu().numpy().transpose((0, 2, 3, 1))
        Yo = vae.decode(Zo_test[sample_indices_tensor]).data.cpu().numpy().transpose((0, 2, 3, 1))
        
        imgs = {"Yv": Yv, "Yr": Yr, "Yo": Yo}
        
        n_seeds = len(results[kernel_name]['mse_per_seed'])
        history = {
            "loss": results[kernel_name]['mse_per_seed'],
            "vs": [[vs.cpu().numpy()[0], vs.cpu().numpy()[1]]] * n_seeds,
            "recon_term": [0.0] * n_seeds,
            "gp_nll": [0.0] * n_seeds,
            "mse_out": results[kernel_name]['mse_per_seed'],
            "mse": [results[kernel_name]['mean_mse']] * n_seeds,
            "mse_val": results[kernel_name]['mse_per_seed'],
        }
    
    callback_gppvae(epoch=n_seeds - 1, history=history, covs=covs, imgs=imgs, ffile=output_file)
    return output_file

print("✅ Callback plot function defined")

In [ ]:
# Generate callback plots
for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and results[kernel_name]['mse_per_seed']:
        print(f"\n📊 Generating callback plot for {kernel_config['display_name']}...")
        filename = f'./notebooks/analysis/task2_callback_{kernel_name}.png'
        result = generate_callback_plot_for_kernel(
            kernel_name, kernel_config, results,
            train_queue, test_queue,
            Dt, Wt, Dtest, Wtest, P, Q, CONFIG['xdim'], device,
            output_file=filename
        )
        if result:
            print(f"   ✅ Saved: {filename}")
            from IPython.display import Image, display
            display(Image(filename=filename))

## 14. Export Results

In [ ]:
# Save detailed results
detailed_results = []

for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and results[kernel_name]['mse_per_seed']:
        for seed_idx, mse in enumerate(results[kernel_name]['mse_per_seed']):
            detailed_results.append({
                'kernel': kernel_config['display_name'],
                'seed': seed_idx,
                'test_mse': mse,
                'variance_ratio': results[kernel_name]['variance_ratios'][seed_idx],
            })

detailed_df = pd.DataFrame(detailed_results)
detailed_df.to_csv('./notebooks/analysis/task2_detailed_results.csv', index=False)
print("📊 Saved: task2_detailed_results.csv")

summary_df.to_csv('./notebooks/analysis/task2_summary.csv', index=False)
print("📊 Saved: task2_summary.csv")

# Save per-view results
per_view_df.to_csv('./notebooks/analysis/task2_per_view_mse.csv', index=False)
print("📊 Saved: task2_per_view_mse.csv")

## 15. Final Summary

In [ ]:
print("\n" + "="*70)
print("📊 TASK 2 (INTERPOLATION) - FINAL SUMMARY")
print("="*70)

print("\n🎯 Research Question:")
print("   Does imposing smoothness via structured kernels improve performance")
print("   when test views lie WITHIN the training range?")

print("\n🏆 Kernel Ranking (by Mean Test MSE):")
print("-"*50)

ranked = sorted(
    [(k, results[k]['mean_mse'], results[k]['std_mse']) 
     for k in results if results[k]['mse_per_seed']],
    key=lambda x: x[1]
)

for rank, (kernel_name, mean_mse, std_mse) in enumerate(ranked, 1):
    display_name = KERNEL_CONFIGS[kernel_name]['display_name']
    medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
    print(f"{medal} {rank}. {display_name:15s}: {mean_mse:.6f} ± {std_mse:.6f}")

# Analysis of interpolation uniformity
print("\n📊 Interpolation Uniformity (variance across test views):")
print("-"*50)
for kernel_name, kernel_config in KERNEL_CONFIGS.items():
    if kernel_name in results and 'mean_mse_per_view' in results[kernel_name]:
        var = np.var(list(results[kernel_name]['mean_mse_per_view'].values()))
        print(f"   {kernel_config['display_name']:15s}: {var:.8f}")

print("\n📁 Generated Files:")
print("-"*50)
print("   - task2_mse_comparison.png")
print("   - task2_per_view_mse.png")
print("   - task2_kernel_matrices.png")
print("   - task2_reconstructions.png")
print("   - task2_detailed_results.csv")
print("   - task2_summary.csv")
print("   - task2_per_view_mse.csv")

print("\n" + "="*70)
print("✅ Analysis Complete!")
print("="*70)